This script is to evaluate the performance of the finetunned LLM with the prompted LLM

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [2]:
# load validation dataset
trajectories = []

In [3]:
modelpath="meta-llama/Meta-Llama-3-8b"

# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    modelpath,    
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(modelpath, use_fast=True)   

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer("i am cat meow meow", return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
logits = outputs.logits

In [5]:
trajectories = [inputs]

In [33]:
import torch
def metric(logits1, logits2):
    return torch.nn.functional.kl_div(torch.nn.functional.log_softmax(logits1, dim=1), torch.nn.functional.softmax(logits2, dim=1))


def remove_prompt_from_trajectory(trajectory, prompt_token_length):
    trajectory_without_prompt = trajectory.copy()
    trajectory_without_prompt["input_ids"] = trajectory_without_prompt["input_ids"][:, prompt_token_length:]
    trajectory_without_prompt["attention_mask"] = trajectory_without_prompt["attention_mask"][:, prompt_token_length:]
    return trajectory_without_prompt

@ torch.no_grad()
def eval_trajectory(finetunned_model, original_model, original_trajectory, prompt_token_length, metric):

    trajectory_without_prompt = remove_prompt_from_trajectory(original_trajectory, prompt_token_length)
    finetunned_logits = finetunned_model(**trajectory_without_prompt).logits

    original_logits = original_model(**original_trajectory).logits[:, -finetunned_logits.shape[1]:, :]

    value = metric(original_logits, finetunned_logits)

    return value


# load fine-tunned model and original model
finetunned_model = model
original_model = model

values = []
for trajectory in trajectories:
    values.append(eval_trajectory(finetunned_model, original_model, trajectory, 0, metric))
print(values)

[tensor(0.2100)]
